In [1]:
import autograd.numpy as np
from autograd import grad

In [20]:
class ORNeuron():
    def __init__(self, n):
        self.n = n
        self.weights = np.zeros(n)
        
    def getWeights(self):
        return self.weights    
    
    def present(self, inputs):
        z = np.sum(np.multiply(inputs, self.weights))
        return 1 - np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[0:len(self.weights)])

In [21]:
class ANDNeuron():
    def __init__(self, n):
        self.weights = np.zeros(n)
#         self.weights = [1, 0, 1, 0, 1]
        
    def getWeights(self):
        return self.weights
        
    def present(self, inputs):
        i = 1.0 - inputs
        i[0] = 1.0
        
        z = np.sum(np.multiply(i, self.weights))
        return np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[0:len(self.weights)])

In [4]:
class CNFNetwork():
    def __init__(self, n, c):
        self.n = n
        self.c = c
        
        d = []
        for i in range(0, c):
            d.append(ORNeuron(n*2 + 1))
        
        self.disjunctions = np.array(d)
        
        if not c == 0:
            self.conjunction = ANDNeuron(c + 1)
        else:
            self.conjunction = ANDNeuron(n*2 + 1)
        
    def getDisjunctions(self):
        return self.disjunctions
    
    def getConjunction(self):
        return self.conjunction
    
    def fowardprop(self, inputs):
        actualIn = self.__convertInputs__(inputs)
        
        if not self.c == 0:
            dout = [1]
            for d in self.disjunctions:
                dout.append(d.present(actualIn))
            
            actualIn = dout
            
        return self.conjunction.present(np.array(actualIn))
    
    def __convertInputs__(self, inputs):
        actual = [1]
        
        for i in inputs:
            actual.append(i)
            actual.append(1-i)
        
        return np.array(actual)
    
    def __repr__(self):
        s = "Disjunctions -> "
        for d in self.disjunctions:
            s += (str(d) + ", ")
            
        s += ("\nConjunction -> " + str(self.conjunction) + "\n")
            
        return s

In [16]:
def SSE(network, data, targets, p=False):
    predictions = predict(network, data)
    
#     if p:
#         print(predictions)
    
    return np.sum(np.power(np.subtract(targets, predictions), 2.0))
    
def predict(network, data):
    t = np.array([network.fowardprop(d) for d in data])
    return t
    

In [26]:
def trainCNFNetwork(data, targets, inputNodes, numC, it=10000):
    network = CNFNetwork(inputNodes, numC)
    print(network)
    print("Initial Loss: ", SSE(network, data, targets, True))
    
    pterb = 0.0001
    
    for i in range(1, it):
        if i%1000 == 0:
            print("Iteration -> " + str(i) + " : " + str(SSE(network, data, targets)))
            
        for d in network.getDisjunctions():
            g = computeNeuronGrad(network, d, pterb, data, targets)
            d.setGrad(g * 0.1)
            
        g = computeNeuronGrad(network, network.getConjunction(), pterb, data, targets)
        
        network.getConjunction().updateWeights(g * 0.1)
        for d in network.getDisjunctions():
            d.applyGrad()
            
    print("Trained Loss: ", SSE(network, data, targets, True))
    return network

def computeNeuronGrad(network, neuron, pterb, data, targets):
    gradient = np.zeros(len(neuron.getWeights()))
    for k in range(0, len(neuron.getWeights())):
        g = np.zeros(len(neuron.getWeights()))
        g[k] = -pterb

        oldSSE = SSE(network, data, targets)
        neuron.updateWeights(g)
        newSSE = SSE(network, data, targets)
        neuron.updateWeights(-g)
                
        gradient[k] = (newSSE - oldSSE)/pterb
        
    return gradient

# NOT

In [7]:
dataNOT = np.array([[1.0], [0.0]])
targetsNOT = np.array([0.0, 1.0])
trainCNFNetwork(dataNOT, targetsNOT, 1, 0)

Disjunctions -> 
Conjunction -> [ 0.  0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.00256401191544
Iteration -> 2000 : 0.00126785197053
Iteration -> 3000 : 0.00084175558195
Iteration -> 4000 : 0.00062993425912
Iteration -> 5000 : 0.000503256413748
Iteration -> 6000 : 0.000418984388068
Iteration -> 7000 : 0.000358880936175
Iteration -> 8000 : 0.000313854296033
Iteration -> 9000 : 0.000278864414562
Trained Loss:  0.000250892296604


Disjunctions -> 
Conjunction -> [ 0.         4.1451229]

# AND
For this we dont neeed any disjunctions

In [8]:
dataAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsAND = np.array([0.0, 0.0, 0.0, 1.0])
trainCNFNetwork(dataAND, targetsAND, 2, 0)

Disjunctions -> 
Conjunction -> [ 0.  0.  0.  0.]

Initial Loss:  3.0
Iteration -> 1000 : 0.00521300404887
Iteration -> 2000 : 0.00255871843019
Iteration -> 3000 : 0.00169422754186
Iteration -> 4000 : 0.00126609639561
Iteration -> 5000 : 0.00101059935924
Iteration -> 6000 : 0.000840864237408
Iteration -> 7000 : 0.000719925065581
Iteration -> 8000 : 0.000629388630652
Iteration -> 9000 : 0.000559072863644
Trained Loss:  0.000502885248704


Disjunctions -> 
Conjunction -> [ 4.1439617  0.         4.1439617  0.       ]

# OR

In [9]:
dataOR = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsOR = np.array([0.0, 1.0, 1.0, 1.0])
trainCNFNetwork(dataOR, targetsOR, 2, 1)

Disjunctions -> [ 0.  0.  0.  0.], 
Conjunction -> [ 0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.0150297778406
Iteration -> 2000 : 0.00672034538128
Iteration -> 3000 : 0.00424944234522
Iteration -> 4000 : 0.00308440448997
Iteration -> 5000 : 0.00241143593159
Iteration -> 6000 : 0.00197491656694
Iteration -> 7000 : 0.00166962036877
Iteration -> 8000 : 0.00144449093602
Iteration -> 9000 : 0.00127182713277
Trained Loss:  0.00113532699449


Disjunctions -> [ 5.32906187  0.          5.32906187  0.        ], 
Conjunction -> [ 3.8528426]

# NAND

In [10]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 1)

Disjunctions -> [ 0.  0.  0.  0.], 
Conjunction -> [ 0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.0150297778406
Iteration -> 2000 : 0.00672034538128
Iteration -> 3000 : 0.00424944234522
Iteration -> 4000 : 0.00308440448997
Iteration -> 5000 : 0.00241143593159
Iteration -> 6000 : 0.00197491656694
Iteration -> 7000 : 0.00166962036877
Iteration -> 8000 : 0.00144449093602
Iteration -> 9000 : 0.00127182713277
Trained Loss:  0.00113532699449


Disjunctions -> [ 0.          5.32906187  0.          5.32906187], 
Conjunction -> [ 3.8528426]

# NOR

In [11]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 0.0, 0.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 2)

Disjunctions -> [ 0.  0.  0.  0.], [ 0.  0.  0.  0.], 
Conjunction -> [ 0.  0.]

Initial Loss:  3.0
Iteration -> 1000 : 0.164350645679
Iteration -> 2000 : 0.128286733334
Iteration -> 3000 : 0.111067878193
Iteration -> 4000 : 0.100277704434
Iteration -> 5000 : 0.0907306733978
Iteration -> 6000 : 0.0201244231162
Iteration -> 7000 : 0.00703532138668
Iteration -> 8000 : 0.00401902263375
Iteration -> 9000 : 0.00276226569966
Trained Loss:  0.00208668761737


Disjunctions -> [ 0.          0.61359876  0.          5.30683655], [ 0.          5.30683655  0.          0.61359876], 
Conjunction -> [ 7.65540844  7.65540844]

# XOR

In [12]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([0.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 2)

Disjunctions -> [ 0.  0.  0.  0.], [ 0.  0.  0.  0.], 
Conjunction -> [ 0.  0.]

Initial Loss:  2.0
Iteration -> 1000 : 0.99999984452
Iteration -> 2000 : 0.061500579224
Iteration -> 3000 : 0.0182706198497
Iteration -> 4000 : 0.0103453313434
Iteration -> 5000 : 0.00713334507329
Iteration -> 6000 : 0.00541386293744
Iteration -> 7000 : 0.00434882132719
Iteration -> 8000 : 0.00362674759388
Iteration -> 9000 : 0.00310608455827
Trained Loss:  0.0027134719969


Disjunctions -> [ 0.          5.55765349  0.          5.55765349], [ 5.55765349  0.          5.55765349  0.        ], 
Conjunction -> [ 3.76454176  3.76454176]

# IMPLIES

In [13]:
dataIMPLYS = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsIMPLYS = np.array([1.0, 1.0, 0.0, 1.0])
trainCNFNetwork(dataIMPLYS, targetsIMPLYS, 2, 1)

Disjunctions -> [ 0.  0.  0.  0.], 
Conjunction -> [ 0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.0150297778406
Iteration -> 2000 : 0.00672034538128
Iteration -> 3000 : 0.00424944234522
Iteration -> 4000 : 0.00308440448997
Iteration -> 5000 : 0.00241143593159
Iteration -> 6000 : 0.00197491656694
Iteration -> 7000 : 0.00166962036877
Iteration -> 8000 : 0.00144449093602
Iteration -> 9000 : 0.00127182713277
Trained Loss:  0.00113532699449


Disjunctions -> [ 0.          5.32906187  5.32906187  0.        ], 
Conjunction -> [ 3.8528426]

# Complicated Formula

## (a AND b) OR (NOT c)
CNF: (a OR (NOT c)) AND (b OR (NOT c))

In [27]:
dataInteresting1 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting1 = np.array([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0])
trainCNFNetwork(dataInteresting1, targetsInteresting1, 3, 2, 30000)

Disjunctions -> [ 0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.], 
Conjunction -> [ 0.  0.  0.]

[ 1.  1.  1.  1.  1.  1.  1.  1.]
Initial Loss:  3.0
Iteration -> 1000 : 0.188245329808
Iteration -> 2000 : 0.140881002185
Iteration -> 3000 : 0.119610350983
Iteration -> 4000 : 0.106739853938
Iteration -> 5000 : 0.0978263887245
Iteration -> 6000 : 0.0911547824558
Iteration -> 7000 : 0.0859022127485
Iteration -> 8000 : 0.0816171925518
Iteration -> 9000 : 0.0780281064768
Iteration -> 10000 : 0.0749601834484
Iteration -> 11000 : 0.0722946107556
Iteration -> 12000 : 0.0696147187254
Iteration -> 13000 : 0.0324331297828
Iteration -> 14000 : 0.00977710251044
Iteration -> 15000 : 0.00530480349437
Iteration -> 16000 : 0.00359329660001
Iteration -> 17000 : 0.00270668498017
Iteration -> 18000 : 0.00216774647356
Iteration -> 19000 : 0.00180639577969
Iteration -> 20000 : 0.00154749473692
Iteration -> 21000 : 0.00135300684304
Iteration -> 22000 : 0.00120163570638
Iteration -> 23000 : 0.00108

Disjunctions -> [ 0.02852533  0.70579929  0.36008377  6.20824543  0.          0.
  7.16114602], [ 0.02852533  6.20824543  0.          0.70579929  0.36008377  0.
  7.16114602], 
Conjunction -> [  0.          10.27165315  10.27165315]

## NOT ( a AND b AND c)
CNF: (NOT a) OR (NOT b) OR (NOT c)

In [23]:
dataInteresting2 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting2 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataInteresting2, targetsInteresting2, 3, 1)

Disjunctions -> [ 0.  0.  0.  0.  0.  0.  0.], 
Conjunction -> [ 0.  0.]

[ 1.  1.  1.  1.  1.  1.  1.  1.]
Initial Loss:  1.0
Iteration -> 1000 : 0.0222633324774
Iteration -> 2000 : 0.00976943121523
Iteration -> 3000 : 0.00612037538354
Iteration -> 4000 : 0.00441688595444
Iteration -> 5000 : 0.00343932422994
Iteration -> 6000 : 0.00280824022053
Iteration -> 7000 : 0.00236848025542
Iteration -> 8000 : 0.00204514531646
Iteration -> 9000 : 0.00179776246315
[ 0.99999952  0.99990526  0.99990526  0.98133717  0.99990526  0.98133717
  0.98133717  0.02361488]
Trained Loss:  0.00160259270776


Disjunctions -> [ 0.          0.          5.29247297  0.          5.29247297  0.
  5.29247297], 
Conjunction -> [ 0.          3.74587818]

## a AND b AND c

In [24]:
dataInteresting3 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting3 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
trainCNFNetwork(dataInteresting3, targetsInteresting3, 3, 3)

Disjunctions -> [ 0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.  0.], 
Conjunction -> [ 0.  0.  0.  0.]

[ 1.  1.  1.  1.  1.  1.  1.  1.]
Initial Loss:  7.0
Iteration -> 1000 : 0.347514605059
Iteration -> 2000 : 0.303266828533
Iteration -> 3000 : 0.280104610647
Iteration -> 4000 : 0.264657474292
Iteration -> 5000 : 0.253193528062
Iteration -> 6000 : 0.243943864912
Iteration -> 7000 : 0.134907926008
Iteration -> 8000 : 0.0231243737434
Iteration -> 9000 : 0.00937534748946
[  7.01200409e-11   1.49865961e-07   4.47159808e-04   2.16997945e-02
   1.02911621e-05   2.19950657e-02   1.92571723e-02   9.34513064e-01]
Trained Loss:  0.00561424155181


Disjunctions -> [ 0.          0.74919618  0.          5.05014699  0.          0.          0.        ], [ 0.          5.0726418   0.          0.72819192  0.          0.          0.        ], [ 0.          0.          0.          0.68057589  0.          5.25965127
  0.        ], 
Conjunction -> [ 0.          7.88601705  7.78745151  7.70734391]